<a href="https://colab.research.google.com/github/Anirudh2465/CDN-optimization-Framework/blob/main/CDN_Optimization_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import networkx as nx
import random
import plotly.graph_objects as go
import time
import threading
import numpy as np

In [ ]:
class Graph:

    graph1 = None

    def __init__(self, n):
        """Initializes a graph with 'n' nodes and no edges."""
        self.graph = nx.Graph()
        self.graph.add_nodes_from(range(n))


    def add_edge(self, u, v, weight):
        """Adds an edge between node 'u' and node 'v' with the given weight."""
        self.graph.add_edge(u, v, weight=weight)


    def print_nodes(self):
        nodes = self.graph.nodes()
        print("Nodes in the graph:")
        for node in nodes:
            print(node)


    def _reconstruct_path(self, source, target, lengths, paths):
        """Reconstruct the path from source to target."""
        if target in paths[source]:
            return paths[source][target]
        else:
            return []


    def display_graph_3d_interactive(graph):
        """Displays the graph in an interactive 3D plot without visible positions/axes."""

        # Generate random positions for each node in 3D
        pos = {node: (random.uniform(-10, 10), random.uniform(-10, 10), random.uniform(-10, 10)) for node in graph.nodes}

        # Extract edge coordinates
        edge_x = []
        edge_y = []
        edge_z = []
        for edge in graph.edges():
            x0, y0, z0 = pos[edge[0]]
            x1, y1, z1 = pos[edge[1]]
            edge_x += [x0, x1, None]
            edge_y += [y0, y1, None]
            edge_z += [z0, z1, None]

        # Extract node coordinates and labels
        node_x = [pos[node][0] for node in graph.nodes]
        node_y = [pos[node][1] for node in graph.nodes]
        node_z = [pos[node][2] for node in graph.nodes]
        node_text = [f'Node {node}' for node in graph.nodes]

        # Create edge trace
        edge_trace = go.Scatter3d(
            x=edge_x, y=edge_y, z=edge_z,
            line=dict(width=1, color='gray'),
            hoverinfo='none',
            mode='lines')

        # Create node trace
        node_trace = go.Scatter3d(
            x=node_x, y=node_y, z=node_z,
            mode='markers+text',
            text=node_text,
            textposition="top center",
            hoverinfo='text',
            marker=dict(size=5, color='blue', opacity=0.8))

        # Create figure and add traces
        fig = go.Figure(data=[edge_trace, node_trace],
                        layout=go.Layout(
                            title="Interactive 3D Graph Visualization",
                            showlegend=False,
                            scene=dict(
                                xaxis=dict(visible=False),  # Hide X-axis
                                yaxis=dict(visible=False),  # Hide Y-axis
                                zaxis=dict(visible=False),  # Hide Z-axis
                                bgcolor="white"  # Background color (adjust if you want!)
                            ),
                            margin=dict(t=100),
                        ))
        fig.show()


    def graph_info(graph):
        # Number of edges
        num_edges = graph.number_of_edges()

        # Check if the graph is connected
        is_connected = nx.is_connected(graph)

        return num_edges, is_connected


    def remove_unwanted_edges(graph, main_server_node):
        """Removes edges connected to main_server_node that are missing the 'weight' attribute."""
        unwanted_edges = []
        for neighbor in list(graph.neighbors(main_server_node)):
            if 'weight' not in graph[main_server_node][neighbor]:
                unwanted_edges.append((main_server_node, neighbor))

        # Remove all unwanted edges from the graph
        graph.remove_edges_from(unwanted_edges)
        return unwanted_edges


    def create_files(number_of_subnets):
        file_list = []
        random.seed(69)
        for i in range(random.randrange(2*number_of_subnets,4*number_of_subnets)):
            file_list.append(f"file{i}")
        return file_list


    def generate_connected_graph(n, edge_probability=0.3):
        g = Graph(n)

        # Create a random connected graph
        while not nx.is_connected(g.graph):
            g.graph.clear()
            for i in range(n):
                for j in range(i + 1, n):
                    if random.random() < edge_probability:
                        weight = random.randint(1, 15)  # Random weight between 1 and 20
                        g.add_edge(i, j, weight)

        return g

In [ ]:
class subnetworks1:
    files_allocation=None
    subnetworks = None
    congestion=None

    def __init__(self, subnetworks):
        self.subnetworks = subnetworks

    def classify_subnetworks(existing_graph, max_size=5):
        """Classifies nodes into subnetworks and returns separate graphs."""
        # Get connected components from the existing graph
        components = list(nx.connected_components(existing_graph))

        # Prepare to hold subnetworks
        subnetworks = []

        # Iterate through each connected component
        for component in components:
            # Convert component to a list
            nodes = list(component)
            # Create subnetworks of max_size
            for i in range(0, len(nodes), max_size):
                subnetwork_nodes = nodes[i:i + max_size]
                if len(subnetwork_nodes) > 1:  # Only create subnetworks with more than 1 node
                    subnetwork_graph = existing_graph.subgraph(subnetwork_nodes).copy()
                    subnetworks.append(subnetwork_graph)

        return subnetworks

    def connect_disconnected_subnetwork(subnetwork, main_graph):
        """Adds edges to connect a disconnected subnetwork and updates the main graph."""
        # Find all connected components in the subnetwork
        components = list(nx.connected_components(subnetwork))

        if len(components) <= 1:
            return subnetwork  # Already connected

        # Connect components and add edges to the main graph
        for i in range(len(components) - 1):
            # Randomly connect a node from the current component to a node in the next component
            node_a = random.choice(list(components[i]))
            node_b = random.choice(list(components[i + 1]))

            weight_random = random.randint(1, 20)
            # Add the edge to both the subnetwork and the main graph
            subnetwork.add_edge(node_a, node_b, weight = weight_random)  # Add an edge to connect the components
            main_graph.add_edge(node_a, node_b, weight = weight_random)  # Also add to the main graph

        return subnetwork


    def print_subnetworks(subnetworks):
        """Prints the nodes and edges of each subnetwork."""
        for i, subnetwork in enumerate(subnetworks):
            print(f"Subnetwork {i + 1}:")
            print(f"Nodes: {list(subnetwork.nodes())}")
            print(f"Edges: {list(subnetwork.edges())}\n")


    def plot_subnetworks_interactive(subnetworks):
        """Plots each subnetwork in an interactive 3D plot using Plotly with colored edges."""
        fig = go.Figure()

        # Fixed offset for plotting subnetworks closer together
        offset = 30
        colors = ['red', 'green', 'blue', 'orange', 'purple', 'cyan', 'magenta', 'yellow', 'gray']

        for i, subnetwork in enumerate(subnetworks):
            # Generate random positions for the nodes within each subnetwork, offset by index
            pos = {node: (np.random.rand() + i * offset, np.random.rand(), np.random.rand()) for node in subnetwork.nodes()}

            # Add edges with different colors for each subnetwork
            edge_color = colors[i % len(colors)]  # Cycle through colors
            for edge in subnetwork.edges():
                x0, y0, z0 = pos[edge[0]]
                x1, y1, z1 = pos[edge[1]]
                fig.add_trace(go.Scatter3d(
                    x=[x0, x1],
                    y=[y0, y1],
                    z=[z0, z1],
                    mode='lines',
                    line=dict(color=edge_color, width=2),  # Color the edges
                    name='Edge'
                ))

            # Add nodes without any markers or color
            xs = [pos[node][0] for node in subnetwork.nodes()]
            ys = [pos[node][1] for node in subnetwork.nodes()]
            zs = [pos[node][2] for node in subnetwork.nodes()]
            fig.add_trace(go.Scatter3d(
                x=xs,
                y=ys,
                z=zs,
                mode='markers+text',  # Just dots for nodes
                text=[f'Node {node}' for node in subnetwork.nodes()],  # Node labels
                marker=dict(size=5, color='black'),  # Simple black dots
                showlegend=False  # Disable legend for simplicity
            ))
        for node in subnetwork.nodes():
                x, y, z = pos[node]
                fig.add_trace(go.Scatter3d(
                    x=[x],
                    y=[y],
                    z=[z],
                    mode='text',
                    text=[str(node)],  # Show the node number
                    textposition="top center",
                    showlegend=False  # Disable legend for simplicity
                ))

        # Update layout
        fig.update_layout(title="Interactive 3D Graph Visualization",
                            showlegend=False,
                            scene=dict(
                                xaxis=dict(visible=False),  # Hide X-axis
                                yaxis=dict(visible=False),  # Hide Y-axis
                                zaxis=dict(visible=False),  # Hide Z-axis
                                bgcolor="white"  # Background color (adjust if you want!)
                            ),
                            margin=dict(t=100),
        )

        # Show the plot
        fig.show()


    def interactive_3d_plot_with_main(graph, subnetworks, main_server_node):
        # Generate a random 3D position for the main server node
        pos = {main_server_node: (0,0,0)}
        offset = 10  # Offset for separating subnetworks visually in the 3D space

        # Node and edge lists for different colors based on subnetworks
        node_x, node_y, node_z, node_colors = [], [], [], []
        edge_x, edge_y, edge_z = [], [], []
        subnetwork_colors = ["blue", "green", "orange", "purple", "pink", "cyan", "yellow"]

        # Plot each subnetwork with a different color
        for i, subnetwork in enumerate(subnetworks):
            # Generate random positions for nodes in this subnetwork
            pos.update({node: (np.random.rand()+1 + i * offset, np.random.rand()+1, np.random.rand()+1) for node in subnetwork.nodes()})
            color = subnetwork_colors[i % len(subnetwork_colors)]

            for node in subnetwork.nodes():
                x, y, z = pos[node]
                node_x.append(x)
                node_y.append(y)
                node_z.append(z)
                node_colors.append(color)

                # Plot edges within the subnetwork
                for neighbor in subnetwork.neighbors(node):
                    x0, y0, z0 = pos[node]
                    x1, y1, z1 = pos[neighbor]
                    edge_x += [x0, x1, None]
                    edge_y += [y0, y1, None]
                    edge_z += [z0, z1, None]

            # Connect the main server node to a random node in each subnetwork
            random_node = random.choice(list(subnetwork.nodes()))
            graph.add_edge(main_server_node, random_node)  # Ensure connection in the main graph
            x0, y0, z0 = pos[main_server_node]
            x1, y1, z1 = pos[random_node]
            edge_x += [x0, x1, None]
            edge_y += [y0, y1, None]
            edge_z += [z0, z1, None]

        # Add main server node in red
        main_x, main_y, main_z = pos[main_server_node]
        node_x.append(main_x)
        node_y.append(main_y)
        node_z.append(main_z)
        node_colors.append("red")

        # Edge trace for plotly
        edge_trace = go.Scatter3d(
            x=edge_x, y=edge_y, z=edge_z,
            line=dict(width=2, color="black"),
            hoverinfo="none",
            mode="lines"
        )

        # Node trace for plotly
        node_trace = go.Scatter3d(
            x=node_x, y=node_y, z=node_z,
            mode="markers+text",
            marker=dict(size=6, color=node_colors, opacity=0.8),
            text=[f"Node {i}" for i in graph.nodes()],
        )

        # Combine traces for interactive plot
        fig = go.Figure(data=[edge_trace, node_trace])
        fig.update_layout(title="Interactive 3D Graph Visualization",
                            showlegend=False,
                            scene=dict(
                                xaxis=dict(visible=False),  # Hide X-axis
                                yaxis=dict(visible=False),  # Hide Y-axis
                                zaxis=dict(visible=False),  # Hide Z-axis
                                bgcolor="white"  # Background color (adjust if you want!)
                            ),
                            margin=dict(t=100),
        )

        fig.show()

    def set_main_server(self, main_server_node, subnetworks):
        # Ensure the main server node is added if not already in the graph
        if main_server_node not in self.graph:
            self.graph.add_node(main_server_node)

        # Connect the main server node to one node in each subnetwork
        for subnetwork in subnetworks:
            # Pick a random node in the subnetwork to connect with the main server
            target_node = random.choice(list(subnetwork))

            # Generate a random weight for the edge
            weight = random.randint(45, 60)

            # Add the edge from the main server node to the chosen node in the subnetwork
            self.add_edge(main_server_node, target_node, weight)


    def assign_files_to_subnetwork(subnetworks, all_files):
        # Ensure there are enough files for the number of subnetworks
        if len(all_files) < len(subnetworks):
            raise ValueError("Number of files must be greater than or equal to the number of subnetworks.")
        # Create a dictionary to store file assignments for each subnetwork (keyed by node group)
        file_assignments = {}
        # First, assign one file to each subnetwork
        random.shuffle(all_files)
        # Iterate over each subnetwork and assign a file
        for subnetwork in subnetworks:
            # Get the nodes in the current subnetwork
            subnetwork_nodes = list(subnetwork.nodes())  # Convert to list of nodes
            node_group_key = tuple(subnetwork_nodes)  # Use tuple as a key
            # Assign the first file to the subnetwork's node group
            file_assignments[node_group_key] = [all_files.pop()]
        # Assign remaining files to subnetworks
        remaining_files = all_files
        for file in remaining_files:
            # Randomly choose a node group to assign the remaining file
            random_node_group = random.choice(list(file_assignments.keys()))
            file_assignments[random_node_group].append(file)
        return file_assignments


    def congestion_setter(graph):
        conjestedval = {}
        for i in graph.nodes():
            conjestedval[i] = 0
        del conjestedval[len(graph.nodes())-1]
        return conjestedval


    def check_node_to_main(subnetwork,graph):
        for node in subnetwork.nodes():
            if graph.has_edge(node,len(graph.nodes())-1):
                return node

    def dijkstra_for_subnet(subnetwork,staring_node):
        results = {}
        length,path = nx.single_source_dijkstra(subnetwork,staring_node,weight='weight')
        for key,value in length.items():
            results[key] = value
        results1= results.copy()
        for i in results:
            if subnetworks1.congestion[i] >= :
                del results1[i]
        return results1

    def best_node_for_datagetting(subnetwork,graph,start_node):
        best_length = 10000
        best_node = None

        if start_node in subnetwork.nodes():
            return start_node

        for node in subnetwork.nodes():
            length = nx.shortest_path_length(graph, start_node, node, weight='weight')
            if length < best_length:
                best_length = length
                best_node = node
        return best_node

    def best_path_finder(graph, starting_node, subnetworks, file):
        print(f"Starting best_path_finder with starting_node: {starting_node} and file: {file}")  # Initial debug

        for subnetwork in subnetworks:

            # Check if the file is in the current subnetwork
            if file in subnetworks1.files_allocation[tuple(subnetwork.nodes())]:

                # Find the best node for data retrieval in the subnetwork
                best_node = subnetworks1.best_node_for_datagetting(subnetwork, graph, starting_node)
                print(f"Best node for data retrieval: {best_node}")  # Debugging best node selection

                # Case: best node is the starting node
                if best_node == starting_node:
                    path = [best_node]
                    length = 0
                    subnetworks1.congestion[best_node] += 1
                    threading.Thread(target=subnetworks1.reduce_congestion, args=(best_node,)).start()
                    print(f"Shortest path found (self-retrieval): {path} with length {length}")  # Debugging self-retrieval path
                    return None

                # Case: best node has congestion < 2
                if subnetworks1.congestion[best_node] < 4:
                    path = nx.shortest_path(graph, source=starting_node, target=best_node, weight='weight')
                    length = nx.path_weight(graph, path, weight='weight')
                    subnetworks1.congestion[best_node] += 1
                    threading.Thread(target=subnetworks1.reduce_congestion, args=(best_node,)).start()
                    print(f"Shortest path found: {path} with length {length}")  # Debugging path with <2 congestion
                    return None

                # Case: best node has congestion >= 2, use Dijkstra within subnetwork
                else:
                    print(f"Best node {best_node} has congestion >= 2, using Dijkstra in subnetwork")  # Debugging congestion condition
                    dijkstra_results = subnetworks1.dijkstra_for_subnet(subnetwork, best_node)
                    print(f"Dijkstra results within subnetwork: {dijkstra_results}")  # Debugging Dijkstra results

                    if len(dijkstra_results) > 0:
                        best_node_1 = min(dijkstra_results, key=dijkstra_results.get)
                        print(f"Best node within subnetwork: {best_node_1}")  # Debugging best node within subnetwork
                        path = nx.shortest_path(graph, source=starting_node, target=best_node, weight='weight')

                        path.append(best_node_1)
                        length = nx.path_weight(graph, path, weight='weight')
                        subnetworks1.congestion[best_node_1] += 1
                        threading.Thread(target=subnetworks1.reduce_congestion, args=(best_node_1,)).start()
                        print(f"Alternative path found: {path} with length {length}")  # Debugging alternative path
                        return None

                    # Case: No alternative node, fallback to main node
                    else:
                        print("No suitable alternative node found, falling back to main node.")  # Debugging fallback
                        node_main = subnetworks1.check_node_to_main(subnetwork, graph)
                        print(f"Fallback main node selected: {node_main}")  # Debugging main node selection
                        path = nx.shortest_path(graph, source=starting_node, target=node_main, weight='weight')
                        path.append(len(graph.nodes()) - 1)
                        length = nx.path_weight(graph, path, weight='weight')
                        print(f"Path to main node found: {path} with length {length}")  # Debugging main node path
                        return None


    def reduce_congestion(node):
        time.sleep(15)  # Wait for 30 seconds
        subnetworks1.congestion[node] = max(0, subnetworks1.congestion[node] - 1)
        print("\n")
        print(f"Congestion reduced for node {node}. Current congestion level: {subnetworks1.congestion[node]}")
        print("\n")



SyntaxError: invalid syntax (<ipython-input-20-a96e95be59e1>, line 266)

In [ ]:
def random_congestion_updater(graph, subnetworks1, max_congestion=2, min_delay=10, max_delay=15):
    while True:
        # Choose a random node to congest
        random_node = random.choice(list(graph.nodes()))

        # Randomly increment the congestion of that node
        random_increase = random.randint(1, max_congestion)
        subnetworks1.congestion[random_node] = subnetworks1.congestion.get(random_node, 0) + random_increase

        # Debugging: print the new congestion level
        print(f"Random congestion added to node {random_node}. New congestion level: {subnetworks1.congestion[random_node]}")

        # Sleep for a random interval before adding congestion again
        time.sleep(random.uniform(min_delay, max_delay))

In [ ]:
def main(number_of_nodes):
    connected_graph = Graph.generate_connected_graph(number_of_nodes)
    subnetworks = subnetworks1.classify_subnetworks(connected_graph.graph)
    subnetworks = [subnetworks1.connect_disconnected_subnetwork(subnetwork, connected_graph.graph) for subnetwork in subnetworks]
    subnetworks1.subnetworks = subnetworks
    #Graph.display_graph_3d_interactive(connected_graph.graph)
    subnetworks1.set_main_server(connected_graph,number_of_nodes,subnetworks)
    #subnetworks1.interactive_3d_plot_with_main(connected_graph.graph,subnetworks,number_of_nodes)
    unwanted_edges_removed = Graph.remove_unwanted_edges(connected_graph.graph, number_of_nodes)
    Graph.graph1 = connected_graph.graph
    files_list = Graph.create_files(len(subnetworks))
    file_distribution = subnetworks1.assign_files_to_subnetwork(subnetworks, files_list)
    file_distribution[(number_of_nodes,)] = Graph.create_files(len(subnetworks))
    subnetworks1.files_allocation = file_distribution
    subnetworks1.congestion = subnetworks1.congestion_setter(connected_graph.graph)

    for i in range(15,20):
      subnetworks1.congestion[i] = 4

    congestion_thread = threading.Thread(target=random_congestion_updater, args=(connected_graph.graph, subnetworks1))
    congestion_thread.start()

    for i in range(50):
      startt = time.time()
      startpt = random.randrange(number_of_nodes)
      file_no = random.randrange(len(files_list))
      file_chosen = files_list[file_no]
      print("\n")
      print("Searching for :", file_chosen, " from node: ", startpt)
      subnetworks1.best_path_finder(connected_graph.graph,startpt,subnetworks,file_chosen)
      endt = time.time()
      print("Time elapsed = ", endt-startt,"\n")
      time.sleep(1)


In [ ]:
if __name__ == "__main__":
    number_of_node = int(input("Enter the number of nodes : "))
    main(number_of_node)

In [ ]:
for subnetwork in subnetworks1.subnetworks:
    print(subnetwork.nodes())

[0, 1, 2, 3, 4]
[5, 6, 7, 8, 9]
[10, 11, 12, 13, 14]
[15, 16, 17, 18, 19]


In [ ]:
subnetworks1.interactive_3d_plot_with_main(Graph.graph1,subnetworks1.subnetworks,20)

Random congestion added to node 15. New congestion level: 6
